In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import re

In [3]:
with open('all_cards', 'rb') as f:
    all_cards = pickle.load(f)
    
with open('names_dates', 'rb') as f:
    names_dates = pickle.load(f)

In [4]:
unique_cards = pd.merge(all_cards, names_dates, on = ['name', 'released_at'])

In [5]:
unique_cards.drop_duplicates(subset = 'name', inplace = True)

In [6]:
pd.set_option('max_columns', None)
unique_cards.sample(1)

,name,released_at,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,reprint,variation,set,set_name,set_type,rarity,flavor_text,artist,edhrec_rank,penny_rank,prices,produced_mana,loyalty,card_faces,life_modifier,hand_modifier,Normal_EDH_Rank,Binary_Rank
20457,veteran's armaments,2008-02-01,{2},2.0,tribal artifact — soldier equipment,"equipped creature has ""whenever this creature ...",NaN,NaN,[],[],[Equip],"{'standard': 'not_legal', 'future': 'not_legal...","[paper, mtgo]",False,True,True,"[nonfoil, foil]",False,False,mor,morningtide,expansion,uncommon,NaN,alan pollack,11290.0,7256.0,"{'usd': '0.19', 'usd_foil': '1.79', 'usd_etche...",NaN,NaN,NaN,NaN,NaN,0.516635,Below_Avg


In [7]:
# unique_cards['Normal_EDH_Rank'].describe()

In [8]:
# words_dict = {}
# words_list = []

In [9]:
# words_list = list(unique_cards['oracle_text'].str.split().explode())

In [10]:
# words_stripped = []
# for e in words_list:
#     e = str(e)
#     e = e.replace("!", "")
#     e = e.replace("?", "")
#     e = e.replace("@", "")
#     e = e.replace("#", "")
#     e = e.replace("$", "")
#     e = e.replace(".", "")
#     e = e.replace(",", "")
#     e = e.replace("(", "")
#     e = e.replace(")", "")
#     e = e.replace(";", "")
#     e = e.replace(":", "")
#     e = e.replace('"', "")
#     e = e.replace("•", "")
#     e = e.replace("—", "")
#     words_stripped.append(e)

In [11]:
# words_stripped = pd.Series(words_stripped)
# words_dict = words_stripped.explode().value_counts().to_dict()
# len(words_dict)

In [12]:
# words_df = pd.DataFrame(data = words_dict.items(), columns = ['word', 'count'])
# words_df.head()

In [13]:
# plt.figure(figsize = (20, 5))
# plt.bar(words_df['word'][0:100], words_df['count'][0:100])
# plt.show()

In [14]:
# do this again but split on '.' for sentances
# get percent of sentence similarity

In [15]:
unique_cards['double_sided'] = unique_cards['card_faces'].notnull().astype("bool")

In [16]:
unique_cards['planeswalker'] = unique_cards['loyalty'].notnull().astype("bool")

In [17]:
unique_cards['makes_mana'] = unique_cards['produced_mana'].notnull().astype("bool")

In [18]:
unique_cards['hand_modify'] = unique_cards['hand_modifier']
unique_cards['hand_modifier'] = unique_cards['hand_modifier'].notnull().astype("bool")

In [19]:
unique_cards['life_modify'] = unique_cards['life_modifier']
unique_cards['life_modifier'] = unique_cards['life_modifier'].notnull().astype("bool")

In [20]:
unique_cards['type_line'] = unique_cards['type_line'].fillna('False')
unique_cards['lands'] = unique_cards['type_line'].apply(lambda x: True if 'land' in x else False)
unique_cards['creature'] = unique_cards['type_line'].apply(lambda x: True if 'creature' in x else False)
unique_cards['enchantment'] = unique_cards['type_line'].apply(lambda x: True if 'enchantment' in x else False)
unique_cards['artifact'] = unique_cards['type_line'].apply(lambda x: True if 'artifact' in x else False)
unique_cards['instant'] = unique_cards['type_line'].apply(lambda x: True if 'instant' in x else False)
unique_cards['sorcery'] = unique_cards['type_line'].apply(lambda x: True if 'sorcery' in x else False)

In [21]:
unique_cards['legendary'] = unique_cards['type_line'].apply(lambda x: True if 'legendary' in x else False)

In [22]:
unique_cards['oracle_text'] = unique_cards['oracle_text'].fillna('')
unique_cards['cmc'] = unique_cards['cmc'].fillna(0.0)

In [23]:
unique_cards['prices'] = unique_cards['prices'].apply(lambda x: x.get('usd'))
unique_cards['prices'] = unique_cards['prices'].fillna(0.0)
unique_cards['prices'] = unique_cards['prices'].apply(lambda x: float(x))

In [24]:
rare_dummies = pd.get_dummies(unique_cards['rarity'],drop_first=True, columns = 'Binary_Rank')
unique_cards = unique_cards.join(rare_dummies)

In [25]:
set_type_dummies = pd.get_dummies(unique_cards['set_type'],drop_first=True, columns = 'Binary_Rank')
unique_cards = unique_cards.join(set_type_dummies)

In [26]:
unique_cards['color_identity'] = unique_cards['color_identity'].apply(lambda x: ['Colorless'] if x == [] else x)
unique_cards['color_identity'] = unique_cards['color_identity'].apply(tuple)
color_dummies = pd.get_dummies(unique_cards['color_identity'],drop_first=True, columns = 'Binary_Rank')
unique_cards = unique_cards.join(color_dummies)

In [29]:
# color_black = pd.get_dummies(unique_cards['color_identity'], columns = 'Binary_Rank')

In [31]:
# color_black = color_black[('B',)]

In [32]:
# with open('color_black', 'wb') as f:
#     pickle.dump(color_black,f)

In [26]:
split_char = '—'

def tribe_splitter(x):
    tribes = x.split(split_char, 1)
    return tribes[1]

In [27]:
unique_cards['Tribes'] = unique_cards['type_line'].apply(lambda x: tribe_splitter(x) if split_char in x else 'no_tribe')

In [28]:
tribe_set = set(['no_tribe'])

def tribe_collecting(x):
    tribe_list = x.split()
    for tribe in tribe_list:
        tribe_set.add(tribe)

In [29]:
unique_cards['Tribes'].apply(lambda x: tribe_collecting(x))

droptribes = ['—', 'the', 'sorcery', 'shadowmoor', "serra’s", 'realm', 'rath', 'rabiah', 'of', 'new', 'nastiest,', 'mercadia',
              'ir', 'iquatana', 'instant', 'innistrad', 'ergamon', 'equilor', 'elemental?', 'dominaria', "bolas’s", 'biggest,',
              'baddest,', 'and/or', '//', 'zendikar', 'xerex', 'vryn', 'valla', 'ulgrotha', 'shandalar', 'segovia', 'regatha',
              'ravnica', 'pyrulea', 'proper', 'phyrexia', 'muraganda', 'mongseng', 'moag', 'mirrodin', 'meditation', 'luvion',
              'lorwyn', 'legendary', 'legend', 'kyneth', 'kolbahan', 'kinshala', 'kephalai', 'karsus', 'kamigawa', 'kaldheim',
              'etiquette', 'card', 'belenon', 'azgol', 'arkhos', 'alara', 'planeswalker', 'creature']

for nontribe in droptribes:
    tribe_set.remove(nontribe)

In [30]:
tribe_set

{'abian',
 'adventure',
 'advisor',
 'aetherborn',
 'ajani',
 'alicorn',
 'alien',
 'ally',
 'aminatou',
 'angel',
 'angrath',
 'antelope',
 'ape',
 'arcane',
 'archer',
 'archon',
 'arlinn',
 'army',
 'art',
 'artifact',
 'artificer',
 'ashiok',
 'assassin',
 'assembly-worker',
 'astartes',
 'atog',
 'aura',
 'aurochs',
 'autobot',
 'avatar',
 'azra',
 'b.o.b.',
 'background',
 'badger',
 'bahamut',
 'barbarian',
 'bard',
 'basilisk',
 'basri',
 'bat',
 'bear',
 'beast',
 'beaver',
 'beeble',
 'beholder',
 'berserker',
 'bird',
 'blood',
 'boar',
 'bolas',
 'brainiac',
 'bringer',
 'brushwagg',
 'bureaucrat',
 'calix',
 'camel',
 'carrier',
 'cartouche',
 'cat',
 'centaur',
 'cephalid',
 'chameleon',
 'chandra',
 'chicken',
 'child',
 'chimera',
 'citizen',
 'clamfolk',
 'class',
 'cleric',
 'cloud',
 'clown',
 'clue',
 'cockatrice',
 'construct',
 'contraption',
 'cow',
 'coward',
 'crab',
 'crocodile',
 'curse',
 'cyborg',
 'cyclops',
 'dack',
 'dakkon',
 'daretti',
 'dauthi',
 'dav

In [31]:
def tribe_matching(x):
    return x.split()

for tribe in tribe_set:
    unique_cards[tribe] = unique_cards['type_line'].apply(lambda x: True if tribe in tribe_matching(x) else False)

C:\Users\mitch\AppData\Local\Temp/ipykernel_16984/3808361938.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  unique_cards[tribe] = unique_cards['type_line'].apply(lambda x: True if tribe in tribe_matching(x) else False)


In [32]:
unique_cards['keywords'] = unique_cards['keywords'].apply(lambda x: ['no_keywords'] if x == [] else x)

In [33]:
keyword_set = set(['no_keywords'])

def keyword_collecting(x):
    for keyword in x:
        keyword_set.add(keyword)
        
unique_cards['keywords'].apply(lambda x: keyword_collecting(x))

0        None
1        None
2        None
3        None
4        None
         ... 
29114    None
29115    None
29116    None
29117    None
29118    None
Name: keywords, Length: 25981, dtype: object

In [34]:
def keyword_matching(x):
    return x.split()

for keyword in keyword_set:
    unique_cards[keyword] = unique_cards['keywords'].apply(lambda x: True if keyword in x else False)

C:\Users\mitch\AppData\Local\Temp/ipykernel_16984/3583498935.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  unique_cards[keyword] = unique_cards['keywords'].apply(lambda x: True if keyword in x else False)


In [35]:
unique_cards.drop(columns = ['no_keywords', 'no_tribe'], axis = 1, inplace = True)

In [36]:
pd.set_option('max_columns', None)
unique_cards.sample(1)

,name,released_at,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,reprint,variation,set,set_name,set_type,rarity,flavor_text,artist,edhrec_rank,penny_rank,prices,produced_mana,loyalty,card_faces,life_modifier,hand_modifier,Normal_EDH_Rank,Binary_Rank,double_sided,planeswalker,makes_mana,hand_modify,life_modify,lands,creature,enchantment,artifact,instant,sorcery,legendary,common,mythic,rare,special,uncommon,archenemy,arsenal,box,commander,core,draft_innovation,duel_deck,expansion,from_the_vault,funny,masterpiece,masters,memorabilia,planechase,premium_deck,promo,spellbook,starter,token,treasure_chest,vanguard,"(B, G)","(B, G, R)","(B, G, R, U)","(B, G, R, U, W)","(B, G, R, W)","(B, G, U)","(B, G, U, W)","(B, G, W)","(B, R)","(B, R, U)","(B, R, U, W)","(B, R, W)","(B, U)","(B, U, W)","(B, W)","(Colorless,)","(G,)","(G, R)","(G, R, U)","(G, R, U, W)","(G, R, W)","(G, U)","(G, U, W)","(G, W)","(R,)","(R, U)","(R, U, W)","(R, W)","(U,)","(U, W)","(W,)",Tribes,pilot,vampire,elminster,eye,elephant,background,gorgon,archon,gold,bat,golem,venser,jeska,avatar,squid,wall,survivor,fungus,fabacin,pangolin,snake,worm,fortification,cat,lolth,trap,angrath,raccoon,orgg,horse,werewolf,hag,pentavite,minotaur,homarid,tibalt,mordenkainen,badger,djinn,elemental,drake,tyvar,siren,incarnation,niko,frog,assassin,monk,turtle,mystic,oko,bureaucrat,gremlin,crab,jaya,noble,rhino,dihada,god,lady,unicorn,calix,lhurgoyf,cockatrice,druid,mountain,treasure,surrakar,centaur,triskelavite,knight,hippogriff,elspeth,lukka,lobster,samut,ooze,goat,walrus,gideon,basilisk,scientist,viashino,mine,yeti,coward,hamster,equipment,volver,kasmina,will,art,fox,rigger,ox,nymph,bolas,undercity,scarecrow,teddy,estrid,salamander,gnome,island,guest,nissa,desert,saproling,flagbearer,sculpture,trilobite,power-plant,curse,gith,kaya,igpay,slith,chimera,kiora,karn,arlinn,wrestler,robot,monkey,garruk,nahiri,land,duck,sphinx,leech,vehicle,berserker,germ,dragon,contraption,atog,myr,townsfolk,dwarf,tasha,kithkin,otter,daretti,mummy,fish,camel,wizard,food,vraska,ranger,yanling,shard,cloud,ferret,rune,scout,dungeon,bahamut,gargoyle,wraith,bard,wrenn,gate,advisor,pegasus,urza,killbot,artificer,skeleton,nautilus,shapeshifter,locus,blood,boar,child,brainiac,cleric,rat,shark,nightmare,elf,insect,pirate,shrine,kraken,kangaroo,minion,antelope,peasant,vivien,phyrexian,satyr,minsc,ape,narset,kavu,wolverine,faerie,juggernaut,scorpion,cephalid,teferi,beast,slug,liliana,griffin,sponge,mongoose,horror,penguin,waiter,elder,devil,shade,hatificer,serra,pest,saheeli,whale,tower,mole,bird,cartouche,ajani,alicorn,davriel,rabbit,head,naga,weird,construct,paratrooper,grandchild,basri,master,astartes,aura,scion,mercenary,octopus,aurochs,mouse,bringer,wolf,huatli,merfolk,nixilis,rebel,oyster,hyena,warrior,sorin,wildfire,samurai,ashiok,yanggu,angel,phoenix,nomad,designer,kirin,b.o.b.,domri,gus,tezzeret,homunculus,kobold,beeble,army,spawn,fractal,citizen,sheep,spike,brushwagg,jace,demon,monger,lammasu,spellshaper,demigod,alien,thopter,gnoll,dakkon,xenagos,dreadnought,egg,zombie,class,mime,ogre,deer,inkling,nephilim,bear,cyclops,azra,thalakos,forest,jackal,zariel,tiefling,hellion,ugin,processor,clue,ral,leviathan,ninja,spy,abian,key,starfish,giant,noggle,cow,swamp,goblin,harpy,inzerva,adventure,teyo,rowan,efreet,ship,wombat,specter,hero,arcane,barbarian,gamer,beaver,masticore,praetor,soldier,freyalise,rogue,dryad,sliver,eldrazi,wurm,human,dauthi,lair,mammoth,lesson,dack,vampyre,illusion,phelddagrif,chameleon,shaman,licid,zubera,windgrace,drone,urza’s,manticore,chicken,metathran,ellywick,donkey,orc,autobot,cyborg,assembly-worker,clown,lizard,sable,snow,dog,serpent,reveler,clamfolk,fire,ally,kaito,spider,lamia,jellyfish,warlock,plains,thrull,elves,szat,koth,elk,nightstalker,aetherborn,reflection,mutant,tamiyo,imp,moonfolk,grist,kor,squirrel,carrier,halfling,spirit,plant,serf,troll,chandra,tentacle,beholder,treefolk,aminatou,vedalken,solta

In [37]:
# drop_cols = ['released_at', 'mana_cost', 'colors', 'legalities', 'games', 'flavor_text', 'set', 'flavor_text', 'edhrec_rank',
#              'penny_rank', 'produced_mana', 'loyalty', 'card_faces', 'Normal_EDH_Rank', 'hand_modify', 'life_modify']

In [38]:
col_names = list(unique_cards.columns)
len(col_names)

782

In [39]:
unique_cards['Binary_Rank'] = unique_cards['Normal_EDH_Rank'].apply(lambda x: 1 if x >= 0.50 else 0)

In [40]:
unique_cards = unique_cards.drop(columns = ['name', 'released_at', 'mana_cost', 'type_line', 'oracle_text', 'power', 'toughness',
                                      'colors', 'color_identity', 'keywords', 'legalities', 'games', 'finishes', 'set',
                                      'set_name', 'set_type', 'rarity', 'flavor_text', 'artist', 'edhrec_rank', 'penny_rank',
                                      'produced_mana', 'loyalty', 'card_faces', 'Tribes', 'Normal_EDH_Rank', 'hand_modify',
                                      'life_modify'])

In [41]:
col_names = list(unique_cards.columns)
len(col_names)

754

In [42]:
with open('unique_cards_cleaned', 'wb') as f:
    pickle.dump(unique_cards,f)

In [43]:
unique_cards.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25981 entries, 0 to 29118
Data columns (total 754 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    cmc                          float64
 1    reserved                     bool   
 2    foil                         bool   
 3    nonfoil                      bool   
 4    reprint                      bool   
 5    variation                    bool   
 6    prices                       float64
 7    life_modifier                bool   
 8    hand_modifier                bool   
 9    Binary_Rank                  int64  
 10   double_sided                 bool   
 11   planeswalker                 bool   
 12   makes_mana                   bool   
 13   lands                        bool   
 14   creature                     bool   
 15   enchantment                  bool   
 16   artifact                     bool   
 17   instant                      bool   
 18   sorcery                 